In [3]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, concat_ws, sha2, row_number, upper, when, trim
from pyspark.sql.window import Window
from delta import *
from datetime import datetime
import logging

In [3]:
process = "coingecko"

# Configuração do logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Definição de variáveis temporais para controle de execução
date_time_exec = datetime.now()                                                # Timestamp da execução atual
data_hora_inicio_file = date_time_exec.strftime("%Y%m%d_%H%M%S")              # Formato para nomes de arquivo: 20240501_130511
data_hora_inicio_format_db = date_time_exec.strftime("%Y-%m-%d %H:%M:%S")      # Formato para banco: 2024-05-01 13:05:11

input_path = f"/mnt/c/Users/arcan/Documents/airflow/data/2-bronze/{process}"
output_path = f"/mnt/c/Users/arcan/Documents/airflow/data/3-silver/{process}"
log_file_path = f"/mnt/c/Users/arcan/Documents/airflow/data/5-logs/2-silver/{process}"

In [4]:
spark = SparkSession.builder \
    .appName("Spark - Silver") \
    .master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate() 

your 131072x1 screen size is bogus. expect trouble
25/06/21 16:18:18 WARN Utils: Your hostname, YanArcanjo resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/21 16:18:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/arcan/Documents/airflow/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/yanarcanjo/.ivy2/cache
The jars for the packages stored in: /home/yanarcanjo/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-72228f4f-1b62-4e3c-89a8-43d5bc791a8a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 256ms :: artifacts dl 14ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

In [9]:
df_bronze = DeltaTable.forPath(spark, input_path).toDF()

df_bronze = df_bronze.withColumn("dt_atualizacao", col("dt_atualizacao").cast("timestamp"))

window_spec = Window.partitionBy("hash_chave").orderBy(col("dt_atualizacao").desc())

df_bronze = df_bronze.withColumn("rank_hash_chave", row_number().over(window_spec)).filter(col("rank_hash_chave") == 1).drop("rank_hash_chave")

if DeltaTable.isDeltaTable(spark, output_path):
    df_silver_original = spark.read.format("delta").load(output_path)
    df_silver_new = df_bronze.join(df_silver_original, df_bronze.hash_colums == df_silver_original.hash_columns, "left_anti")
else:
    df_silver_new = df_bronze

df_silver_new.show(truncate=False)

25/06/21 15:00:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+---------------------+------------------------+----------+---------------------+------------------------+--------------------+-------------+-----------------------+--------+--------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------------+--------+-----------+---------------------+--------------------------------+---------------+--------------------+--------------------------------------------------+---------------------+---------------------------+-------+--------------------+-------------+-----------------------------------------------------------------------------------+-------------------+-----------+------------------------------------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+
|ath     |ath_change_percentage|ath_date     

In [10]:
string_cols = [field.name for field in df_silver_new.schema.fields if str(field.dataType) == "StringType()" and field.name not in ['hash_columns','hash_chave']]
for col_name in string_cols:
    #df_sz_new = df_sz_new.withColumn(col_name, trim(upper(when(col(col_name).isNotNull(), col(col_name)).otherwise(''))))
    df_silver_new = df_silver_new.withColumn(col_name, trim(upper(when(col(col_name) == '', None).otherwise(col(col_name)))))

df_silver_new = df_silver_new.drop('key_log_file_path','dt_atualizacao','dt_ingestao','nm_arquivo_origem')

#Adiciona coluna de ID de log
df_silver_new = df_silver_new.withColumns({
    "key_log_file_path": lit(f'{log_file_path}_{data_hora_inicio_file}'),
    "dt_atualizacao": lit(data_hora_inicio_format_db),
    "dt_ingestao": lit(datetime.now().strftime("%Y-%m-%d")),
    "nm_arquivo_origem": lit(input_path),
    })

In [11]:
if DeltaTable.isDeltaTable(spark, output_path):
    df_silver = DeltaTable.forPath(spark, output_path)

    df.silver.alias("silver")\
        .merge(df_silver_new.alias("bronze"), "silver.hash_chave = bronze.hash_chave") \
        .whenNotMatchedInsertAll()\
        .whenMatchedUpdateAll()\
        .execute()

else:
    df_silver_new.write.format('delta').mode("append").option("mergeSchema", "true").save(output_path)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, row_number, upper, when, trim
from pyspark.sql.window import Window
from delta import *
import sys
from datetime import datetime
import logging

# process = sys.argv[1]
process = "coingecko"

# Configuração do logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Definição de variáveis temporais para controle de execução
date_time_exec = datetime.now()                                                # Timestamp da execução atual
data_hora_inicio_file = date_time_exec.strftime("%Y%m%d_%H%M%S")              # Formato para nomes de arquivo: 20240501_130511
data_hora_inicio_format_db = date_time_exec.strftime("%Y-%m-%d %H:%M:%S")      # Formato para banco: 2024-05-01 13:05:11

input_path = f"/mnt/c/Users/arcan/Documents/airflow/data/2-bronze/{process}"
output_path = f"/mnt/c/Users/arcan/Documents/airflow/data/3-silver/{process}"
log_file_path = f"/mnt/c/Users/arcan/Documents/airflow/data/5-logs/2-silver/{process}"

logging.info("Iniciando criação da Spark Session")

spark = SparkSession.builder \
    .appName("Spark - Silver") \
    .master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate() 

logging.info("Criação da Spark Session Finalizada")

logging.info(f'Inicio do processamento para {process} as {data_hora_inicio_format_db}')

try:
    # Leitura da Delta table e guarda ordem de colunas
    logging.info(f"Lendo delta bronze -> {input_path}...")
    df_bronze = DeltaTable.forPath(spark, input_path).toDF()
    initial_column_order = df_bronze.columns 

    #Seleciona os registros mais recentes de cada hash chave
    logging.info('Selecionando os registros mais recentes de cada hash chave')
    df_bronze = df_bronze.withColumn("dt_atualizacao", col("dt_atualizacao").cast("timestamp"))
    window_spec = Window.partitionBy("hash_chave").orderBy(col("dt_atualizacao").desc())
    df_bronze = df_bronze.withColumn("rank_hash_chave", row_number().over(window_spec)).filter(col("rank_hash_chave") == 1).drop("rank_hash_chave")

    # Pega apenas as linhas que existem alguma alteração
    if DeltaTable.isDeltaTable(spark, output_path):
        logging.info('Filtrando os dados que necessitam ser atualizados ou inseridos')
        df_silver_original = spark.read.format("delta").load(output_path)
        df_silver_new = df_bronze.join(df_silver_original, df_bronze.hash_colums == df_silver_original.hash_columns, "left_anti")
    else:
        logging.info('A tabela silver não existe, será criada com todos os registros.')
        df_silver_new = df_bronze

    # Identifica as colunas do tipo string e aplica a transformação
    string_cols = [field.name for field in df_silver_new.schema.fields if str(field.dataType) == "StringType()" and field.name not in ['hash_columns','hash_chave']]
    for col_name in string_cols:
        df_silver_new = df_silver_new.withColumn(col_name, trim(upper(when(col(col_name) == '', None).otherwise(col(col_name)))))

    df_silver_new = df_silver_new.drop('key_log_file_path','dt_atualizacao','dt_ingestao','nm_arquivo_origem')

    #Adiciona coluna de ID de log
    df_silver_new = df_silver_new.withColumns({
        "key_log_file_path": lit(f'{log_file_path}_{data_hora_inicio_file}'),
        "dt_atualizacao": lit(data_hora_inicio_format_db),
        "dt_ingestao": lit(datetime.now().strftime("%Y-%m-%d")),
        "nm_arquivo_origem": lit(input_path),
        })

    if DeltaTable.isDeltaTable(spark, output_path):
        logging.info('Realizando Merge para atualizar silver')
        df_silver = DeltaTable.forPath(spark, output_path)
        # Atualiza dados da silver com merge
        df_silver.alias("silver")\
            .merge(df_silver_new.alias("bronze"), "silver.hash_chave = bronze.hash_chave") \
            .whenNotMatchedInsertAll()\
            .whenMatchedUpdateAll()\
            .execute()

    else:
        # Inserindo todos os registros com append, pois é uma tabela nova
        logging.info('Inserindo todos os registros pois é uma nova tabela')
        df_silver_new.write.format('delta').mode("append").option("mergeSchema", "true").save(output_path)

except Exception as e:
    logging.error(f"Erro na execução: {e}")

2025-06-21 15:30:51,020 - INFO - Iniciando criação da Spark Session
your 131072x1 screen size is bogus. expect trouble
25/06/21 15:30:56 WARN Utils: Your hostname, YanArcanjo resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/21 15:30:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/arcan/Documents/airflow/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/yanarcanjo/.ivy2/cache
The jars for the packages stored in: /home/yanarcanjo/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-48f33895-0a79-4ef7-b119-3bfda63c9f2b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 239ms :: artifacts dl 19ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

In [11]:
log_df = spark.read.format("delta").load("/mnt/c/Users/arcan/Documents/airflow/data/5-logs/2-silver/coingecko")

In [13]:
log_df.show(truncate=False)

+-----------------------------------------------------------------------------------+--------------------+----------+-------------------+-------------------+-------+-------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+----------------+---------------+--------------------+-------------------+-------------------+-------------------+---------------------+-------------------+-----------------------+---------------------+--------------------+-------------------+-------------------------+-----------------------+
|id_log_file_path                                                                   |process_name        |date_run  |start_time_script  |end_time_script    |status |msg                                                          |input_path                                                  |output_path                                                 |records_

In [15]:
silver_df = spark.read.format("delta").load("/mnt/c/Users/arcan/Documents/airflow/data/3-silver/coingecko")

In [16]:
silver_df.show(truncate=False)

+--------+---------------------+------------------------+----------+---------------------+------------------------+--------------------+-------------+-----------------------+--------+--------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------------+--------+-----------+---------------------+--------------------------------+---------------+--------------------+--------------------------------------------------+---------------------+---------------------------+-------+--------------------+-------------+----------------------------------------------------------------+----------------------------------------------------------------+-----------------------------------------------------------------------------------+-------------------+-----------+------------------------------------------------------------+
|ath     |ath_change_percentage|ath_date                |atl       |atl_cha